# IS470 Data Visualisation

## Lab Test Qns : 2A, 2B, 2C 

NB: increasing levels of difficulty

## Summary using 2C as example - A simple demo

* import pandas (Python Library)
* generate basic statistics to understand data: 
    * Quantitative: no. of successful compiles/runs/correct results
    * Qualitative: overview of examples of compile errors, runtime errors, and wrong/correct outputs
* Plan of action
    * Unsupervised Learning: perform clustering on textual data (generate character-tuples)
    * Analyse similarities with Multinomial Logistic Regression
    * use Elbow Plot to perform dimensionality reduction on the data

In [1]:
import pandas as pd

In [2]:
def stats(sample):
    print "=== Total Students ===\n", sample.count(), "\n"

    feature_cols = ['COMPILE','RUN','CORRECT']
    for feature in feature_cols:
        print "===", feature, "Stats===\n", sample[feature].value_counts(),"\n"

## Overview of Quantitative Data (mostly binary)

In [3]:
url = 'all_summary2C.txt'
train_all2C = pd.read_csv(url)
stats(train_all2C)
train_all2C.head(4)

=== Total Students ===
ID         261
COMPILE    261
CPERR      261
CPLINE     261
RUN        261
RTERR      261
RTLINE     261
RESULT     178
CORRECT    185
dtype: int64 

=== COMPILE Stats===
1    185
0     76
Name: COMPILE, dtype: int64 

=== RUN Stats===
1    135
-     76
0     50
Name: RUN, dtype: int64 

=== CORRECT Stats===
0    148
1     37
Name: CORRECT, dtype: int64 



,ID,COMPILE,CPERR,CPLINE,RUN,RTERR,RTLINE,RESULT,CORRECT
0,G1/Qi.huang.2014,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN
1,G1/Weilun.fan.2015,1,-,-,1,-,-,0A0B0C0D 0A0B0C0D 2A0B0C0D 0A1B0C0D 0A0B3C0D 2...,1
2,G1/Wsyeo.2015,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN
3,G1/avery.chong.2015,0,missing method body or declare abstract,public static int countNumFighters(String str);,-,-,-,NaN,NaN


## Overview of Quantitative Data

### Compilation Errors : Extract error type, Line at which compile error occurred

In [4]:
# Print (a sample of) instances where compilation failed
# specifically of the error "cannot find symbol": most common error, around 40/76
# Note: this error is most frequent. And occurs at the same line for almost every student who committed the error

train_all2C[train_all2C.COMPILE == 0][train_all2C.CPERR == "cannot find symbol"].head(5)

/Users/behminyan/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,ID,COMPILE,CPERR,CPLINE,RUN,RTERR,RTLINE,RESULT,CORRECT
0,G1/Qi.huang.2014,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN
2,G1/Wsyeo.2015,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN
10,G1/jh.tan.2015,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN
29,G1/xinying.tan.2015,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN
43,G2/deborah.sim.2015,0,cannot find symbol,"System.out.println(Q2PartC.countNumFighters(""""));",-,-,-,NaN,NaN


In [5]:
# an (sample) list of other compile errors. Note similarities across students

train_all2C.drop(['RUN','RTERR','RTLINE','RESULT'],axis=1)[train_all2C.COMPILE == 0][train_all2C.CPERR != "cannot find symbol"].tail(11)

,ID,COMPILE,CPERR,CPLINE,CORRECT
130,G4/kwchoo.2015,0,';' expected,"String a = ""<->""",NaN
153,G5/chiahui.mah.2015@sis.smu.edu.sg,0,not a statement,char,NaN
164,G5/kyusang.lee.2015,0,<identifier> expected,factory.getAllAvailableModels();,NaN
173,G5/theresa.lee.2015,0,illegal start of type,return toReturn;,NaN
176,G5/xinyi.lim.2015,0,')' expected,System.out.print( tff.retrieve('>');,NaN
187,G6/carine.ng.2015,0,<identifier> expected,"System.out.(""expected :2A1B3C0D"");",NaN
213,G6/vera.low.2015,0,'void' type not allowed here,"System.out.println(Q2PartC.countNumFighters(""""));",NaN
214,G6/wildy.cheng.2015,0,invalid method declaration; return type required,public static String (String s ){,NaN
217,G6/xinyao.nie.2015,0,';' expected,int i = 0,NaN
245,G7/mnnya.2015_labTest2,0,non-static method getAllAvailableModels() cann...,String models = TieFighterFactory.getAllAvaila...,NaN


### Runtime Errors : Common error types (null pointers, index out of bounds)

In [6]:
# a (sample) list of runtime errors. E.g. Null-Pointer Exceptions and Index-Out-of-Bounds most common

train_all2C.drop(['COMPILE','CPERR','CPLINE'], axis=1)[train_all2C.COMPILE == 1][train_all2C.RUN == "0"].head(4)

,ID,RUN,RTERR,RTLINE,RESULT,CORRECT
5,G1/dnn.nguyen.2012,0,"Exception in thread ""main"" java.lang.StringInd...",at java.lang.String.charAt(String.java:658),NaN,0
8,G1/jbachtiar.2015@sis.smu.edu.sg,0,"Exception in thread ""main"" java.lang.NullPoint...",at Q2PartC.countNumFighters(Q2PartC.java:67),0A0B0C0D,0
13,G1/jityang.lim.2015,0,"Exception in thread ""main"" java.lang.StringInd...",at java.lang.String.substring(String.java:1963),0A0B0C0d 0A0B0C0D,0
15,G1/jqchong.2015,0,"Exception in thread ""main"" java.lang.StringInd...",at java.lang.String.substring(String.java:1967),0A0B0C0D 0A0B0C0D 2A0B0C0D,0


### Incorrect Outputs (for submissions without compile or runtime errors)

In [7]:
# Sample of 15 incorrect outputs - Note similarties for select test cases
# Results for different test cases are concatenated for now for easy visualisation

train_all2C.drop(['COMPILE','CPERR','CPLINE','RUN','RTERR','RTLINE'],axis=1)[train_all2C.RUN == "1"][train_all2C.CORRECT == 0].head(15)

,ID,RESULT,CORRECT
6,G1/ervin.chong.2015,0A0B0C0D 0A0B0C0D 1A0B0C0D 0A1B0C0D 0A0B0C0D 1...,0
7,G1/gary.soong.2015,0A0B0C0D 0A0B0C0D 3A0B0C0D 0A0B0C0D 16A0B14C15...,0
9,G1/jeryl.soh.2015,0A0B0C0D 0A0B0C0D 2A0B0C0D 0A0B0C0D 0A0B0C0D 2...,0
11,G1/jiaying.ang.2015,,0
16,G1/jxzhang.2015,0A0B0C0D 0A0B0C0D 2A0B0C0D,0
19,G1/lingwei.au.2015,A A A A A A,0
21,G1/manfred.loh.2015,0A0B0C0D 0A0B0C0D 1A0B0C0D 0A1B0C0D 1A0B3C3D 1...,0
24,G1/nacheau.2015,0A0B0C0D 0A0B0C0D,0
26,G1/shivania.2014,0A0B0C0D 0A0B0C0D 1A0B0C0D 0A1B0C0D 2A0B1C1D 2...,0
28,G1/weixuan.tay.2015,ABCD ABCD ABCD ABCD ABCD ABCD,0


## Other Statistics (from Qns 2A & 2B)

In [8]:
# note that there are far less errors for earlier questions (very simple). Compile/Runtime errors are few.

url = 'all_summary2A.txt'
train2A = pd.read_csv(url)
stats(train2A)
train2A.head()

=== Total Students ===
ID         261
COMPILE    261
CPERR      261
CPLINE     261
RUN        261
RTERR      261
RTLINE     261
RESULT     259
CORRECT    260
dtype: int64 

=== COMPILE Stats===
1    260
0      1
Name: COMPILE, dtype: int64 

=== RUN Stats===
1    259
0      1
-      1
Name: RUN, dtype: int64 

=== CORRECT Stats===
1    231
0     29
Name: CORRECT, dtype: int64 



,ID,COMPILE,CPERR,CPLINE,RUN,RTERR,RTLINE,RESULT,CORRECT
0,G1/Qi.huang.2014,0,cannot find symbol,String s = tieFighter.retrieve('A');,-,-,-,NaN,NaN
1,G1/Weilun.fan.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->*<->*<->*<->,1
2,G1/Wsyeo.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->*<->*<->*<->,1
3,G1/avery.chong.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->*<->*<->*<->,1
4,G1/clare.yip.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->*<->*<->*<->,1


In [9]:
url = 'all_summary2B.txt'
train2B = pd.read_csv(url)
stats(train2B)
train2B.head()

=== Total Students ===
ID         261
COMPILE    261
CPERR      261
CPLINE     261
RUN        261
RTERR      261
RTLINE     261
RESULT     254
CORRECT    255
dtype: int64 

=== COMPILE Stats===
1    255
0      6
Name: COMPILE, dtype: int64 

=== RUN Stats===
1    254
-      6
0      1
Name: RUN, dtype: int64 

=== CORRECT Stats===
1    152
0    103
Name: CORRECT, dtype: int64 



,ID,COMPILE,CPERR,CPLINE,RUN,RTERR,RTLINE,RESULT,CORRECT
0,G1/Qi.huang.2014,1,-,-,1,-,-,,0
1,G1/Weilun.fan.2015,1,-,-,1,-,-,<-> **<->*** <-|-><-|-> ***<-|-><-|->*** *<-|...,1
2,G1/Wsyeo.2015,1,-,-,1,-,-,<-> **<->*** <-|-><-|-> ***<-|-><-|->*** *<-|...,1
3,G1/avery.chong.2015,1,-,-,1,-,-,<-> **<->** <-|-><-|-> ***<-|-><-|->**** *<-|...,0
4,G1/clare.yip.2015,1,-,-,1,-,-,<-> **<->*** <-|-><-|-> ***<-|-><-|->*** *<-|...,1


In [10]:
# train2A['CORRECT'] = [x  == 0 for x in train2A.CORRECT]

train2A[train2A.COMPILE == 1][train2A.CORRECT == 0].head(15) # note that Incorrect results have many similarities

,ID,COMPILE,CPERR,CPLINE,RUN,RTERR,RTLINE,RESULT,CORRECT
11,G1/jiaying.ang.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->,0
18,G1/keiwen.tam.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->,0
37,G2/aliciatan.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->*<->*<->*<->,0
40,G2/clarey.liow.2015,1,-,-,1,-,-,## <-|->*<-|->* <->*<->*<->*<->*<->*,0
43,G2/deborah.sim.2015,1,-,-,1,-,-,#<-># <-|->*<-|-> <->*<->*<->*<->*<->,0
46,G2/eslynn.choo.2013,1,-,-,1,-,-,## <-|->*<-|->* <->*<->*<->*<->*<->*,0
49,G2/hztan.2015,1,-,-,1,-,-,## <-|->*<-|-> <->*<->,0
61,G2/singwei.lau.2015,1,-,-,1,-,-,<->## <-|->*<-|-> <->*<->*<->*<->*<->,0
69,G2/yg.chen.2015,1,-,-,1,-,-,<->## <-|->*<-|-> <->*<->*<->*<->*<->,0
72,G2/zccui.2014,1,-,-,1,-,-,## <-|->*<-|->*<-|-> <->*<->*<->*<->*<->*<->,0
